### 인터페이스와 프로토콜
- 프로토콜이라는 개념과 인터페이스라는 개념은 동적언어와 정적언어에서 객체가 역할을 수행하는 방법에 관해 서로 약간 다르게 정의하고 구현하는 것에 차이가 있다.
- 하지만 크게 보면 프로토콜과 인터페이스는 대동소이 하고 언어적 특성과 방향성에서 조금씩 달라진다고 보면 될 것 같다.
- 즉, 프로토콜은 인터페이스이지만 비공식적이다.  
- python에서는 X같은 객체, X 프로토콜, X 인터페이스 라는 말이 전부 동의어이다.
- 사실 시퀀스가 동작하려면 \__iter\__()와 \__init\__()이 필요한데 \__getitem__()만 구현해도 충분히 시퀀스 처럼 동작한다.

### 시퀀스 처럼 동작하는 객체
- 하지만 \__getitem__()만 구현하면 이것은 불변 시퀀스 처럼 동작한다.
- 가변 시퀀스 처럼 동작하게 하려면  \__setitem__()을 구현해야 한다.

### 몽키패칭 (Monkey patch)
- 런타임에 함수를 구현해서 특별 메소드에 대입해 버리면 런타임에 동작을 변경할 수 있다.
- 이것을 몽키패칭 이라고 한다.
- 원래 게릴라 패치 였다가 발음이 비슷한 고릴라 패치로 바뀌고 고릴라가 험악해서 몽키패치로 바뀌었다.
- 몽키패칭은 언발에 오줌누기 식으로 땜빵용으로 봐야할 듯 하다.
- 코틀린과 스위프트에도 이런식으로 메소드 확장하는 방법이 있다.

### ABC 상속
- ABC란? Abstract Base Class의 줄임말로 Python에서 객체지향 인터페이스를 관리할 수 있도록 만든 메타클래스이다.
- ABC클래스를 사용하면 python 답지 않지만 좀 더 엄격(strict)하게 메소드 오버라이딩을 강요할 수 있다.
- ABC를 사용하는것에 대한 장점
  - ABC를 상속받는 최상위 base class는 그 자체로 인스턴스가 될 수 없다. 아직 abstract method를 구현하지 않았기 때문이다.
  - 기존 NotImplementedError를 발생시키는 방법은 그 메소드가 호출되었을 때만 알 수 있다. 하지만 ABC를 이용하는 경우에는 인스턴스화 시점에 바로 에러가 발생한다.
- 기존 ABC를 제대로 사용하는 것 만으로도 엄청난 효율을 볼 수 있다.
- 함부로 ABC를 새로 만들지 말자. 유지보수의 어려움과 잘못된 설계에서 비롯된 버그가 난무할 것이다.
- 

### ABC 상속 및 사용하기

In [11]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  
        self._cards[position] = value

    def __delitem__(self, position):  
        del self._cards[position]

    def insert(self, position, value):  
        self._cards.insert(position, value)

In [12]:
deck = FrenchDeck2()

- 가정 상황
> 웹사이트나 모바일 앱세서 광고를 무작위 순으로 보여주어야 하지만, 광고 목록에 들어 있는 광고를 모두 보여주기 전까지는 같은 광고를 반복하면 안 된다.

In [8]:
import abc

class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable):
        " 아이템을 컨테이너 안에 넣는다"

    @abc.abstractmethod
    def pick(self):
        "컨테이너 안에서 무작위로 아이템 하나를 꺼내서 반환한다"

    def loaded(self):
        "컨테이너 안에 아이템이 하나 이상 들어 있으면 True를 반환한다"
        return bool(self.inspect())

    def inspect(self):
        """
        내용물을 변경하지 않고 컨테이너 
        안에 들어 있는 항목으로 부터 
        만든 정렬된 튜플을 반환한다
        """
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [10]:
class Fake(Tombola):
    def pick(self):
        return 13

f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

In [11]:
t = Tombola()

TypeError: Can't instantiate abstract class Tombola with abstract methods load, pick

다른 데코레이터와 같이 쌓을 수 있지만 abstractmethod가 항상 제일 안쪽에 있어야 한다. 즉 def와 abstractmethod 사이에 아무것도 올 수 없다!!!!!!

In [ ]:
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def func1(cls, ...):
        pass

In [ ]:
import random

class BingoCage(Tombola): 

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  
        self._items = []
        self.load(items)  

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  

    def pick(self):  
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  
        self.pick()


class LotteryBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)  

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))  
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position)  

    def loaded(self):  
        return bool(self._balls)

    def inspect(self):  
        return tuple(sorted(self._balls))

register 사용법

In [12]:
from random import randrange


@Tombola.register  
class TomboList(list):  

    def pick(self):
        if self:  
            position = randrange(len(self))
            return self.pop(position)  
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  

    def loaded(self):
        return bool(self)  

    def inspect(self):
        return tuple(sorted(self))

Tombola.register(TomboList)

__main__.TomboList

In [14]:
Tombola.__subclasses__()

[__main__.Fake, __main__.Fake]

### subclasshook 특별 메서드
- subclasshook 특별 메서드는 해당 객체의 상속 여부 및 특별 메소드의 구현 여부 까지 융통성 있게 검사한다

In [ ]:
class Struggle:
    def __len__(self):
        return 23

from collections import abc
isinstance(Struggle(), abc.Sized)

In [ ]:
from abc import ABCMeta, abstractmethod

class Sized(metaclasss=ABCMeta):
    __slots__ = ()
    
    @abstractmethod
    def __len__(self):
        return 0
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplementedError

- ABC를 배웠다면 모든곳에 ABC를 써먹고 싶겠지만 ABC를 사용하기 전에 먼저 덕파이핑으로 문제를 해결해 보는것이 먼저다.
- 먼저 파이썬의 융통성을 받아들여야 한다.